## Part 2

1. If you haven't installed it yet please do so. You can simply follow these steps
2. To include Bokeh in your notebooks you can follow the Bokeh: Using with Jupyter guide. Come back to this one when you need it.
3. We aim to give you a gentle start with Bokeh and I am going to include more example code than usual in the follwing.



In [2]:
from bokeh.plotting import figure, output_file, show

output_file("output.html")

p = figure()
p.line(x=[1, 2, 3], y=[4,6,2])

show(p)

In [1]:
from bokeh.plotting import figure, output_file, show

# create a figure object
p = figure(width=300, height=300, tools="pan,reset,save")

# add a Circle renderer to this figure
p.circle([1, 2.5, 3, 2], [2, 3, 1, 1.5], radius=0.3, alpha=0.5)

# specify how to output the plot(s)
output_file("foo.html")

# display the figure
show(p)

In [4]:
from bokeh.plotting import figure, show

p = figure(width=400, height=400)

# add a square renderer with a size, color, and alpha
p.diamond_dot([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="olive", alpha=0.5)

# show the results
show(p)

In [115]:
import pandas as pd 
df = pd.read_csv("../../../datasat.csv")
df['Date'] = pd.to_datetime(df['Date'])
# slice the dataframe to get data between 2010 and 2017
df = df[(df['Date'].dt.year >= 2010) & (df['Date'].dt.year <= 2017)]
focuscrimes = set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])
df = df[df['Category'].isin(focuscrimes)]


#### Data prep
1. Take the data for the period of 2010-2017 and group it by hour-of-the-day.


In [116]:
df['HourOfDay'] = df['Time'].str.strip().str[0:2]


In [117]:
# group the dataframe by "Category" and "HourOfDay", then calculate the count for each group
crime_hourly_counts = df.groupby(['Category', 'HourOfDay']).size().reset_index(name='count')

# calculate the total count for each crime category
crime_category_counts = df.groupby(['Category']).size().reset_index(name='total_count')

# merge the two dataframes to get the total count for each row
crime_hourly_counts = pd.merge(crime_hourly_counts, crime_category_counts, on='Category')

# calculate the hourly percentage of each crime type
crime_hourly_counts['hourly_percentage'] = crime_hourly_counts['count'] / crime_hourly_counts['total_count']


columns = ['Category', 'HourOfDay', 'hourly_percentage']
focusData =  pd.DataFrame(crime_hourly_counts, columns=columns)
print(focusData)

        Category HourOfDay  hourly_percentage
0        ASSAULT        00           0.055468
1        ASSAULT        01           0.049745
2        ASSAULT        02           0.044837
3        ASSAULT        03           0.023267
4        ASSAULT        04           0.014025
..           ...       ...                ...
331  WEAPON LAWS        19           0.065000
332  WEAPON LAWS        20           0.054413
333  WEAPON LAWS        21           0.052851
334  WEAPON LAWS        22           0.062918
335  WEAPON LAWS        23           0.054760

[336 rows x 3 columns]


In [118]:
# Pivot the dataframe
pivoted_focusData = focusData.pivot_table(index='HourOfDay', columns='Category', values='hourly_percentage')

# Display the pivoted dataframe
print(pivoted_focusData)


Category    ASSAULT  BURGLARY  DISORDERLY CONDUCT  \
HourOfDay                                           
00         0.055468  0.040191            0.052282   
01         0.049745  0.027653            0.038354   
02         0.044837  0.031432            0.032569   
03         0.023267  0.032765            0.018642   
04         0.014025  0.029379            0.014999   
05         0.011857  0.025644            0.060210   
06         0.015573  0.022892            0.119777   
07         0.022212  0.032284            0.101564   
08         0.033324  0.048688            0.074566   
09         0.035732  0.044997            0.053139   
10         0.040480  0.041786            0.040926   
11         0.042750  0.039973            0.036212   
12         0.053690  0.051462            0.030641   
13         0.046454  0.033660            0.039854   
14         0.046810  0.037898            0.028498   
15         0.052371  0.043686            0.031069   
16         0.052371  0.050698            0.027

1. First, let's convert our Pandas Dataframe to Bokeh ColumnDataSource:

In [119]:
from bokeh.models import ColumnDataSource,Legend
from bokeh.io import output_notebook, show
from bokeh.palettes import Category10
import seaborn as sns

source = ColumnDataSource(data=pivoted_focusData)
## it is a standard way to convert your df to bokeh
output_notebook()


Loading BokehJS ...

2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:

In [167]:
# Define a figure with title and axis labels
p = figure(x_range=source.data['HourOfDay'], title="Hourly Percentage by Category",x_axis_label='Hour of the Day',width =1200)
colo = sns.color_palette('viridis', len(source.data['HourOfDay'])).as_hex()

3. Now we are going to add the bars. In order to do so, we will use vbar (see the guide for help):

In [168]:
bar ={} # to store vbars
items=[]


### here we will do a for loop:
for indx,category  in enumerate(pivoted_focusData.columns):
    bar[category] =p.vbar(x='HourOfDay', 
    top=category ,
    source=source,
    muted=True, 
    muted_alpha=0.05,
    fill_alpha=1.9,
    color=colo[indx],
    width=0.7)
    items.append((category, [bar[category]]))


4. The last thing to do is to make your legend interactive and display the figure:


In [169]:
legend = Legend(items=items)
p.add_layout(legend, 'left') 
p.legend.click_policy="mute" 
#test
### assigns the click policy (you can try to use ''hide'
show(p)
#displays your plot